In [ ]:
#import names from excel
import re, pandas
input_file = "David_Test.xlsx"
directory = r"C:\Users\linge\Desktop\Vaccine Fairy\\"

df = pandas.read_excel(io=directory+"names\\"+ input_file)
df.rename(columns={"First Name":"First","Last Name":"Last","Phone Number":"Phone","Zip Code":"Zip","Email Address":"Email"},inplace=True)
df["Zip"] = df["Zip"].astype(str)
df.Zip = df.Zip.str.zfill(5)

In [ ]:
##Setup initial driver session
import json, time, requests, pandas
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import bs4, datetime, pickle
import glob, os
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

url = "https://www.google.com/"
options = Options()
#options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument(r"load-extension=C:\Users\linge\AppData\Local\Google\Chrome\User Data\Profile 1\Extensions\cjpalhdlnbpafiamejdnhcphjbkeiagm\1.34.0_14")
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
options.add_experimental_option('prefs', prefs)
options.add_argument('--kiosk-printing')
driver = webdriver.Chrome(r"C:\Users\linge\chromedriver.exe", options=options)
executor_url = driver.command_executor._url
session_id = driver.session_id
driver.create_options()
driver.get(url)



In [ ]:
##adds tabs
for x in range (1,len(df.index)+1):
    driver.execute_script("window.open('https://www.cvs.com/vaccine/intake/store/cvd-schedule.html?icid=coronavirus-lp-vaccine-sd-statetool');")
    driver.switch_to.window(driver.window_handles[-1]) 
    time.sleep(0.5)
for x in range (1,len(df.index)+1):
    driver.switch_to.window(driver.window_handles[-x]) 
    row = df.iloc[x-1]
    
    time.sleep(2)
    ##screening questions
    driver.find_element_by_css_selector("[for = 'q7_2']").click()
    time.sleep(.5)
    driver.find_element_by_css_selector("[for = 'q8_2']").click()
    time.sleep(.5)
    driver.find_element_by_css_selector("[for = 'q9_2']").click()
    time.sleep(.5)
    driver.find_element_by_css_selector(".btn-control").click()

    #next page
    time.sleep(.5)
    driver.find_element_by_css_selector("[for='customRadio_1']").click()
    time.sleep(.5)
    driver.find_element_by_css_selector("[type='submit']").click()
    time.sleep(1)
    driver.find_element_by_css_selector("[formcontrolname='jurisdiction']").send_keys(row["StateLong"])
    time.sleep(.5)
    driver.find_element_by_css_selector("[formcontrolname='jurisdiction']").send_keys(Keys.ENTER)
    time.sleep(.5)
    driver.find_element_by_css_selector("[type='submit']").click()
    time.sleep(1)

    #eligibility page
    driver.find_element_by_id("q1_0").send_keys("65") ##age
    time.sleep(.5)
    driver.find_element_by_css_selector("[for='q20']").click()     ##change for other eligiblity
    time.sleep(.5)
    driver.find_element_by_id("qconsent").click()    
    time.sleep(.5)
    driver.find_element_by_css_selector("[type='button']").click()     
    time.sleep(.5)

    #confirm
    driver.find_element_by_css_selector("[type='submit']").click()     


    #driver.switch_to.alert().dismiss()


In [ ]:
##search locations - AT 12:00
def grab_slot():
    for x in range (1,len(df.index)+1):
        driver.switch_to.window(driver.window_handles[-x]) 
        ###search page
        row = df.iloc[x-1]
        try:
            driver.find_element_by_id("address")
        except NoSuchElementException:
            print(row["Last"] + " booked or error")
            continue
        try: 
            driver.find_element_by_css_selector(".row.patient-form-heading")
            print(row["Last"] + " booked or error")
            continue
        except NoSuchElementException:
            pass
        #if not driver.find_element_by_id("address").get_attribute("value"):
        #driver.find_element_by_id("address").send_keys(row["City"] + ", " + row["State"])    ###location - replace with row["Zip"][:-2]
        driver.find_element_by_id("address").clear()   ###location - replace with row["Zip"]

        driver.find_element_by_id("address").send_keys(row["City"] + ", " + row["State"])    ###location - replace with row["Zip"][:-2]
        time.sleep(0.5)
        driver.find_element_by_css_selector("[aria-label='submit search']").click()     

        time.sleep(4)

        ##select appts
        #first dose
        try:
            driver.find_element_by_id("availableTimes0").click()
        except NoSuchElementException:
            try:
                driver.find_element_by_css_selector(".suggest-add-list-item").click()
                time.sleep(2)
            except NoSuchElementException:
                print("No appts found/error for " + row.First + " "+ row.Last + " near " + row.Zip)
                continue

        time.sleep(3)
        try:
            driver.find_element_by_css_selector(".slot-btn").click()
        except NoSuchElementException:
            print("No appts found/error for " + row.First + " "+ row.Last + " near " + row.Zip)
            continue
        time.sleep(0.5)
        driver.find_element_by_css_selector(".submit.btn-control").click()
        time.sleep(4)
        #second dose
        try: 
            driver.find_element_by_css_selector(".slot-btn").click()
        except NoSuchElementException:
            print("No second dose found for " + row.First + " "+ row.Last + " near " + row.Zip)
            continue    
        time.sleep(0.5)
        driver.find_element_by_css_selector(".submit.btn-control").click()
        
        print("found slot for "+row.Last)
        #book_appt()

In [ ]:
grab_slot()
print("Last checked at: ", datetime.now().time())

In [ ]:
##enter patient info
def book_appt_1():
    #info
    driver.find_element_by_id("firstName").send_keys(row.First)  #first name
    time.sleep(0.5)
    driver.find_element_by_id("lastName").send_keys(row.Last) #last name
    time.sleep(0.5)
    driver.find_element_by_id("dob").send_keys(row.DOB.strftime("%m%d%Y")) #DOB - ddmmyyyy
    time.sleep(0.5)
    if row.Gender == "Female":
        driver.find_element_by_css_selector("[for='customRadio_F']").click()  #Gender - F for female, M for male
    else:
        driver.find_element_by_css_selector("[for='customRadio_M']").click()  #Gender - F for female, M for male
    time.sleep(0.5)
    driver.find_element_by_id("address").send_keys(row.Address) #address
    time.sleep(0.5)
    if len(str(row.Zip))<5 :
        row.Zip = "0" + str(row.Zip)
    driver.find_element_by_id("zip").send_keys(row.Zip) #zip
    time.sleep(0.5)
    driver.find_element_by_id("city").send_keys(row.City) #city
    time.sleep(0.5)
    driver.find_element_by_id("state").send_keys(row.State) #state
    time.sleep(0.5)
    driver.find_element_by_id("email").send_keys(row.Email) #email
    time.sleep(0.5)
    if isinstance(row.Phone, str):
        driver.find_element_by_id("phoneNumber").send_keys(re.sub("[^0-9]", "", row.Phone)) # ##########
    else:
        driver.find_element_by_id("phoneNumber").send_keys(row.Phone) # ##########
    time.sleep(0.5)
    

In [ ]:
def book_appt_2():
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(1.5)
    #no insurance
    driver.find_element_by_css_selector("[for='customRadio_7']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(0.5)
    #first consent
    driver.find_element_by_css_selector("[for='consent']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(2)
    #allergies
    driver.find_element_by_css_selector("[for='q2_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("[for='q3_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("[for='q4_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("[for='q5_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("[for='q6_3']").click()
    time.sleep(0.1)
    driver.find_element_by_css_selector("[for='q7_3']").click()
    time.sleep(0.1)
    driver.find_element_by_css_selector("[for='q11_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("[for='q12_3']").click()
    time.sleep(0.1)
    driver.find_element_by_css_selector("[for='q13_3']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(0.5)
    #race
    driver.find_element_by_css_selector("[for='qrace_2131-1']").click()
    time.sleep(0.1)
    driver.find_element_by_css_selector("[for='qethnicity_U']").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(0.5)
    #confirm
    driver.find_element_by_css_selector(".btn-control").click()
    time.sleep(0.5)
    #second consent
    driver.find_element_by_css_selector("[for='consent']").click()
    time.sleep(0.1)
    driver.find_element_by_css_selector(".btn-control").click()
    print("appointment booked for " + row.First + " " + row.Last)
    #driver.execute_script('window.print();')


In [ ]:
#book reserved appointments
x=1
for row in df.itertuples(index=False):
    driver.switch_to.window(driver.window_handles[-x]) 
    try: 
        driver.find_element_by_id("firstName")
    except NoSuchElementException:
        x=x+1
        continue
    book_appt_1()
    #input("Please verify information")
    book_appt_2()
    x=x+1

#create datetime column
#hours = df.groupby(["Name",'First Name',"Surname","DATE"],as_index=False)["Hours"].sum() ##sums all - issue if not labor
#hours = hours.values.tolist()
